In [ ]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

In [ ]:
# Slice Dimension Attributes defined in the plugin. Please check all queries and replace <KEY HERE> with a valid name.
# For example: If slice is defined by Version.[Version Name] and Time.[Month]
# input_df = ibpl Select ([Version].[Version Name].[<KEY HERE>] * [Time].[Month].[<KEY HERE>] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;
#                             update <KEY HERE> to valid names
# input_df = ibpl Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Month].[January] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;

_input_df = "select (Version.[Version Name]*Product.[Product].[<KEY HERE>]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
_predict_df = "select (Version.[Version Name]*Product.[Product].[<KEY HERE>]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
O9DataLake.register("input_df",DataSource.LS, ResourceType.IBPL, _input_df)
O9DataLake.register("predict_df",DataSource.LS, ResourceType.IBPL, _predict_df)

In [ ]:
"""
exec plugin instance [CsvMGIP1_CsvMGIP2_CsvMGPrimDf_CsvMGOP1_Slice] for measures {[DPSellOutUnitsActuals]} using scope ([Product].[Product].Filter(#.Name in {[196426],[208821]}) * [Time].[Day] * [Version].[Version Name].[CurrentWorkingView]);

"""
import logging
from sklearn import tree

logger = logging.getLogger('o9_logger')

logger = logging.getLogger('o9_logger')
logger.info('logging azure creds..')
logger.info(os.environ)
out_df = None

if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    # predict_keys = predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]',                               'Location.[Location]']]
    # predict_keys["Product.[Product]"] = pd.to_numeric(predict_keys["Product.[Product]"], downcast='integer')
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    #out_df = pd.concat([predict_keys, out_df_part], axis=1)
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)
    print('####################################### This is a test message #################################################')
    log_msg = '####################################### This is another test message #################################################'
    logger.info(log_msg)

else:
    out_df = None